In [4]:
# run the following to use gpu
#import os
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import theano
from theano import tensor as T

x = T.dscalar('x')
y = x ** 2
gy = T.grad(y, x)

In [3]:
import scipy
import numpy as np

shared_w = theano.shared(np.array(np.array([3.0, -3.0]).astype(np.float64), dtype=np.float32))

inputs_x = np.array([[2, 1], [0.3, 0.5]]).astype(np.float32)
inputs_y = np.array([[1, 0]]).astype(np.float32)

output = T.dot(shared_w, inputs_x)


if 0:
    # mse loss
    loss = T.mean(T.sum((output-inputs_y)**2, axis=1))

if 1:
    # hinge loss
    target2 = T.switch(inputs_y>0, inputs_y, -1)
    margin = 1-target2* output
    loss = T.mean(margin* (margin>0))


grad = T.grad(loss, shared_w)

f_loss = theano.function([], loss)
f_grad = theano.function([], grad)

print 'before opt:', f_loss()

# non-theano functions that can be called by optimizer
def eval_loss(x0):
    shared_w.set_value(x0.astype(np.float32))
    return f_loss().astype('float64')


def eval_grad(x0):
    shared_w.set_value(x0.astype(np.float32))
    return np.array(f_grad()).flatten().astype('float64')


w_opt = np.array([[3.0, -3.0]]).astype(np.float64)
opt_result = scipy.optimize.fmin_l_bfgs_b(eval_loss, w_opt, fprime=eval_grad, maxfun=40)
print 'after opt:', opt_result[1]

before opt: 1.25
after opt: 0.0
